In [1]:
import json
import numpy as np
import pandas as pd

from pathlib import Path

In [2]:
df = pd.read_json("./results/multinli_transforms_bertscore_dev_matched.jsonl", lines=True).set_index('pairID')
df_erwin = pd.read_json("./annotations/raw/annotator_1.json").set_index('pairID')
df_consolidation = pd.read_json("./annotations/consolidation/admin.json").set_index('pairID')
df_consolidation = df_consolidation[["id", "text", "orig_prem", "orig_hyp", "orig_label", "label", "Comments"]]

In [4]:
df_joined = df.join(df_erwin)
df_joined = df_joined[df_joined['gold_label'] != '-']
df_joined = df_joined.dropna()

In [24]:
df_joined_groups = df_joined.groupby('style')
df_joined_emb_sample = df_joined_groups.get_group('emb').sample(167)
df_joined_pir_sample = df_joined_groups.get_group('pir').sample(166)
df_joined_sha_sample = df_joined_groups.get_group('sha').sample(167)
df_joined_sampled = pd.concat([df_joined_emb_sample, df_joined_pir_sample, df_joined_sha_sample])

In [25]:
for x, y in df_joined_sampled.groupby('style'):
    print(x, len(y))
    for i, j in y.groupby('genre'):
        print(i, len(j))
    print()

emb 167
fiction 33
government 36
slate 37
telephone 31
travel 30

pir 166
fiction 32
government 33
slate 37
telephone 32
travel 32

sha 167
fiction 32
government 36
slate 28
telephone 38
travel 33



In [32]:
df_joined_sampled_labels = df_joined_sampled[['label']]
df_joined_sampled_labels['label'] = df_joined_sampled_labels['label'].apply(lambda x: x[0])

/tmp/ipykernel_70939/2060080551.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_joined_sampled_labels['label'] = df_joined_sampled_labels['label'].apply(lambda x: x[0])


In [34]:
save_path = Path('./annotations/gold.json')
df_joined_sampled_labels.to_json(save_path)